# BINANCE AUTOMATED TRADING WITH ML PREDICTIONS

In [1]:
#Import necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime
from datetime import date
import plotly.offline as py #visualization
import plotly.graph_objs as go #visualization
import plotly.subplots as tls #visualization
import plotly.figure_factory as ff #visualization
py.init_notebook_mode(connected=True) #visualization
from scipy import stats
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
sns.set(style="white")
import time
from time import sleep
import math
import random
from numpy import arange
from numpy import argmax
from sklearn.preprocessing import StandardScaler

In [81]:
#I define this round down function in order not to have problems to put orders with less than my available amount

def round_decimals_down(number:float, decimals:int=4):
    """
    Returns a value rounded down to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.floor(number)

    factor = 10 ** decimals
    return math.floor(number * factor) / factor

## 1. DATA

In [2]:
import requests

market = 'ETHUSDT'
tick_interval = '1h'

url = 'https://api.binance.com/api/v3/klines?symbol='+market+'&interval='+tick_interval
data = requests.get(url).json()

data = pd.DataFrame(data)

data.columns = ['Open_Time','Open','High','Low','Close',
               'Volume','Close_Time','Quote_Asset_Volume',
               'Trades','Taker_buy_Volume','Taker_buy_asset_volume','Out']

data['Open_Time'] = data.Open_Time.apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
data['Close_Time'] = data.Close_Time.apply(lambda x: datetime.datetime.fromtimestamp(x/1000))
data['Open'] = data['Open'].astype(float)
data['High'] = data['High'].astype(float)
data['Low'] = data['Low'].astype(float)
data['Quote_Asset_Volume'] = data['Quote_Asset_Volume'].astype(float)
data['Close'] = data['Close'].astype(float)
data['Taker_buy_Volume'] = data['Taker_buy_Volume'].astype(float)
data['Taker_buy_asset_volume'] = data['Taker_buy_asset_volume'].astype(float)
data['Volume'] = data['Volume'].astype(float)
data['Trades'] = data['Trades'].astype(int)

#data['Return'] = data['High']/data['Open']

#Here I set the conditions!
data['Return'] = np.where((data['High']/data['Open'] > 1.02) &
                          (data['Close']/data['Open'] > 1.015) &
                          (data['Low']/data['Open'] > 0.97), True, False)


data = data.drop(['Out'], axis = 1)

data = data.reset_index()

df = data.copy()

#data['Return_1'] = 0
#data['Return_1'] = data['Return'].ffill()

data_1 = data.copy()
data_1['index'] = data_1['index']-1
data_1 = data_1.drop(['Open_Time','Open','High','Low','Close',
               'Volume','Close_Time','Quote_Asset_Volume',
               'Trades','Taker_buy_Volume','Taker_buy_asset_volume'], axis = 1)

data = pd.merge(data,data_1, how = 'inner', on = 'index')

data = data.drop(['Return_x', 'index'], axis = 1)

data.columns = ['Open_Time','Open','High','Low','Close',
               'Volume','Close_Time','Quote_Asset_Volume',
               'Trades','Taker_buy_Volume','Taker_buy_asset_volume', 'Return']

data.head()

,Open_Time,Open,High,Low,Close,Volume,Close_Time,Quote_Asset_Volume,Trades,Taker_buy_Volume,Taker_buy_asset_volume,Return
0,2021-12-29 13:00:00,3795.05,3799.32,3770.42,3784.34,7866.3086,2021-12-29 13:59:59.999,2.976231e+07,21349,3652.6774,1.382078e+07,False
1,2021-12-29 14:00:00,3784.33,3784.76,3748.87,3754.46,7197.7953,2021-12-29 14:59:59.999,2.711219e+07,19345,3406.1131,1.282904e+07,False
2,2021-12-29 15:00:00,3754.47,3777.03,3745.00,3753.65,9586.1162,2021-12-29 15:59:59.999,3.605702e+07,20683,4395.9200,1.653329e+07,False
3,2021-12-29 16:00:00,3753.65,3755.83,3712.84,3725.68,9801.4623,2021-12-29 16:59:59.999,3.658612e+07,26369,4732.8865,1.766420e+07,False
4,2021-12-29 17:00:00,3725.69,3768.51,3718.07,3746.22,9488.2592,2021-12-29 17:59:59.999,3.553895e+07,24097,4706.9848,1.763012e+07,False


I scale features used to predict

In [3]:
def Zscore(df):
    score=zscore(np.array(df))
    return np.abs(score[len(score)//2])

In [4]:
se=StandardScaler()

In [5]:
data['Open_norm']=se.fit_transform(data.Open.values.reshape(-1, 1))
data['High_norm']=se.fit_transform(data.High.values.reshape(-1, 1))
data['Low_norm']=se.fit_transform(data.Low.values.reshape(-1, 1))
data['Close_norm']=se.fit_transform(data.Close.values.reshape(-1, 1))
data['Quote_Asset_Volume_norm']=se.fit_transform(data.Quote_Asset_Volume.values.reshape(-1, 1))
data['Trades_norm']=se.fit_transform(data.Trades.values.reshape(-1, 1))
data['Taker_buy_Volume_norm']=se.fit_transform(data.Taker_buy_Volume.values.reshape(-1, 1))
data['Taker_buy_asset_volume_norm']=se.fit_transform(data.Taker_buy_asset_volume.values.reshape(-1, 1))

In [7]:
#data = data[data['Return_y'] == True]
#data
#I see how many of the candlesticks did actually fill the conditions
data.groupby('Return').count()

,Open_Time,Open,High,Low,Close,Volume,Close_Time,Quote_Asset_Volume,Trades,Taker_buy_Volume,Taker_buy_asset_volume,Open_norm,High_norm,Low_norm,Close_norm,Quote_Asset_Volume_norm,Trades_norm,Taker_buy_Volume_norm,Taker_buy_asset_volume_norm
Return,,,,,,,,,,,,,,,,,,,
False,493,493,493,493,493,493,493,493,493,493,493,493,493,493,493,493,493,493,493
True,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6


In [8]:
data_see = data[data['Return'] == True]
data_see

,Open_Time,Open,High,Low,Close,Volume,Close_Time,Quote_Asset_Volume,Trades,Taker_buy_Volume,Taker_buy_asset_volume,Return,Open_norm,High_norm,Low_norm,Close_norm,Quote_Asset_Volume_norm,Trades_norm,Taker_buy_Volume_norm,Taker_buy_asset_volume_norm
39,2021-12-31 04:00:00,3738.46,3748.34,3687.00,3721.88,11817.6224,2021-12-31 04:59:59.999,4.386023e+07,26241,6294.9974,2.336069e+07,True,1.207742,1.183919,1.077520,1.150335,-0.040706,-0.121181,-0.063287,0.052007
215,2022-01-07 12:00:00,3208.20,3209.66,3064.42,3131.69,62334.0102,2022-01-07 12:59:59.999,1.954928e+08,107790,28214.2028,8.854184e+07,True,-0.813448,-0.875220,-1.278246,-1.099927,3.945555,3.935762,3.765030,3.601108
244,2022-01-08 17:00:00,3028.75,3055.06,3003.30,3031.62,22372.1694,2022-01-08 17:59:59.999,6.780989e+07,44543,12765.0411,3.870278e+07,True,-1.497458,-1.466188,-1.509516,-1.481471,0.588905,0.789317,1.066744,0.887380
287,2022-01-10 12:00:00,3018.96,3032.40,2981.03,3006.26,40379.5084,2022-01-10 12:59:59.999,1.213584e+08,69871,20391.3803,6.129941e+07,True,-1.534774,-1.552807,-1.593783,-1.578163,1.996639,2.049348,2.398728,2.117762
311,2022-01-11 12:00:00,3115.33,3158.56,3110.75,3133.56,21964.9648,2022-01-11 12:59:59.999,6.886031e+07,45910,11657.8969,3.655412e+07,True,-1.167441,-1.070553,-1.102939,-1.092797,0.616520,0.857323,0.873375,0.770386
381,2022-01-14 10:00:00,3211.39,3230.75,3203.39,3210.15,10100.8664,2022-01-14 10:59:59.999,3.251572e+07,21121,4779.1069,1.538496e+07,True,-0.801289,-0.794602,-0.752400,-0.800777,-0.338942,-0.375894,-0.328046,-0.382271


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 499 entries, 0 to 498
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Open_Time                    499 non-null    datetime64[ns]
 1   Open                         499 non-null    float64       
 2   High                         499 non-null    float64       
 3   Low                          499 non-null    float64       
 4   Close                        499 non-null    float64       
 5   Volume                       499 non-null    float64       
 6   Close_Time                   499 non-null    datetime64[ns]
 7   Quote_Asset_Volume           499 non-null    float64       
 8   Trades                       499 non-null    int64         
 9   Taker_buy_Volume             499 non-null    float64       
 10  Taker_buy_asset_volume       499 non-null    float64       
 11  Return                       499 non-null    

## 2. DATA PRE PROCESSING

In [10]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

## 3. ML MODEL

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score,roc_curve #, scorer
from sklearn.metrics import f1_score
import statsmodels.api as sm
from sklearn.metrics import precision_score,recall_score
from yellowbrick.classifier import DiscriminationThreshold

In [12]:
data.head()

,Open_Time,Open,High,Low,Close,Volume,Close_Time,Quote_Asset_Volume,Trades,Taker_buy_Volume,Taker_buy_asset_volume,Return,Open_norm,High_norm,Low_norm,Close_norm,Quote_Asset_Volume_norm,Trades_norm,Taker_buy_Volume_norm,Taker_buy_asset_volume_norm
0,2021-12-29 13:00:00,3795.05,3799.32,3770.42,3784.34,7866.3086,2021-12-29 13:59:59.999,2.976231e+07,21349,3652.6774,1.382078e+07,False,1.423446,1.378793,1.393171,1.388481,-0.411326,-0.364551,-0.524783,-0.467440
1,2021-12-29 14:00:00,3784.33,3784.76,3748.87,3754.46,7197.7953,2021-12-29 14:59:59.999,2.711219e+07,19345,3406.1131,1.282904e+07,False,1.382585,1.323137,1.311628,1.274555,-0.480995,-0.464247,-0.567847,-0.521440
2,2021-12-29 15:00:00,3754.47,3777.03,3745.00,3753.65,9586.1162,2021-12-29 15:59:59.999,3.605702e+07,20683,4395.9200,1.653329e+07,False,1.268767,1.293588,1.296985,1.271467,-0.245845,-0.397683,-0.394972,-0.319744
3,2021-12-29 16:00:00,3753.65,3755.83,3712.84,3725.68,9801.4623,2021-12-29 16:59:59.999,3.658612e+07,26369,4732.8865,1.766420e+07,False,1.265642,1.212550,1.175295,1.164823,-0.231935,-0.114813,-0.336119,-0.258166
4,2021-12-29 17:00:00,3725.69,3768.51,3718.07,3746.22,9488.2592,2021-12-29 17:59:59.999,3.553895e+07,24097,4706.9848,1.763012e+07,False,1.159067,1.261020,1.195085,1.243138,-0.259464,-0.227842,-0.340643,-0.260022


In [13]:
data_model = data[(['Open_Time','Close_Time','Open_norm',
                  'High_norm','Low_norm','Close_norm','Trades_norm',
                    'Taker_buy_Volume_norm','Taker_buy_asset_volume_norm',
                   'Return'])]

In [14]:
data_model.head()

,Open_Time,Close_Time,Open_norm,High_norm,Low_norm,Close_norm,Trades_norm,Taker_buy_Volume_norm,Taker_buy_asset_volume_norm,Return
0,2021-12-29 13:00:00,2021-12-29 13:59:59.999,1.423446,1.378793,1.393171,1.388481,-0.364551,-0.524783,-0.467440,False
1,2021-12-29 14:00:00,2021-12-29 14:59:59.999,1.382585,1.323137,1.311628,1.274555,-0.464247,-0.567847,-0.521440,False
2,2021-12-29 15:00:00,2021-12-29 15:59:59.999,1.268767,1.293588,1.296985,1.271467,-0.397683,-0.394972,-0.319744,False
3,2021-12-29 16:00:00,2021-12-29 16:59:59.999,1.265642,1.212550,1.175295,1.164823,-0.114813,-0.336119,-0.258166,False
4,2021-12-29 17:00:00,2021-12-29 17:59:59.999,1.159067,1.261020,1.195085,1.243138,-0.227842,-0.340643,-0.260022,False


In [15]:
pd.options.display.max_columns = None

In [16]:
#separando en train y test
train, test = train_test_split(data_model, test_size = .2 , random_state = random.randint(1,10))

if test['Return'].sum() == 0:
    train, test = train_test_split(data_model, test_size = .2 , random_state = random.randint(1,10))
else:
    print('GO')

#Sacando la columna con DriverID, la del target (Churn) y la del mes
Open = ['Open_Time']
Close = ['Close_Time']
target_col = ['Return']
cols    = [i for i in data_model.columns if i not in Open + Close + target_col]
train_X = train[cols]
train_Y = train[target_col]
test_X  = test[cols]
test_Y  = test[target_col]

GO


In [17]:
#Function attributes#

#dataframe     - processed dataframe
#Algorithm     - Algorithm used 
#training_x    - predictor variables dataframe(training)
#testing_x     - predictor variables dataframe(testing)
#training_y    - target variable(training)
#training_y    - target variable(testing)
#cf - ["coefficients","features"](cooefficients for logistic regression, features for tree based models)
#threshold_plot - if True returns threshold plot for model

def churn_prediction(algorithm, training_x, testing_x, training_y, testing_y, cols, cf, threshold_plot) :
    
    #model
    algorithm.fit(training_x, training_y)
    predictions   = algorithm.predict(testing_x)
    probabilities = algorithm.predict_proba(testing_x)
    
    #coeffs
    if   cf == "coefficients" :
        coefficients  = pd.DataFrame(algorithm.coef_.ravel())
    elif cf == "features" :
        coefficients  = pd.DataFrame(algorithm.feature_importances_)
        
    column_df     = pd.DataFrame(cols)
    coef_sumry    = (pd.merge(coefficients, column_df, left_index= True, right_index= True, how = "left"))
    coef_sumry.columns = ["coefficients", "features"]
    coef_sumry    = coef_sumry.sort_values(by = "coefficients", ascending = False)
    
    print (algorithm)
    print ("\n Classification report : \n",classification_report(testing_y, predictions))
    print ("Accuracy   Score : ",accuracy_score(testing_y, predictions))
    
    #confusion matrix, en el eje x van los reales y en el eje y los que predice el algoritmo
    conf_matrix = confusion_matrix(testing_y, predictions)
    
    #roc_auc_score
    model_roc_auc = roc_auc_score(testing_y, predictions) 
    print ("Area under curve (AUC) : ",model_roc_auc,"\n")
    fpr,tpr,thresholds = roc_curve(testing_y,probabilities[:,1])
    
    #plot confusion matrix
    trace1 = go.Heatmap(z = conf_matrix ,
                        x = ["False","True"],
                        y = ["False","True"],
                        showscale  = False,colorscale = "Picnic",
                        name = "matrix")
    
    #plot roc curve
    trace2 = go.Scatter(x = fpr,y = tpr,
                        name = "Roc : " + str(model_roc_auc),
                        line = dict(color = ('rgb(22, 96, 167)'), width = 2))
    trace3 = go.Scatter(x = [0,1],y=[0,1],
                        line = dict(color = ('rgb(205, 12, 24)'), width = 2,
                        dash = 'dot'))
    
    #plot coeffs
    trace4 = go.Bar(x = coef_sumry["features"], y = coef_sumry["coefficients"],
                    name = "coefficients",
                    marker = dict(color = coef_sumry["coefficients"],
                                  colorscale = "Picnic",
                                  line = dict(width = .6,color = "black")))
    
    #subplots
    fig = tls.make_subplots(rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]],
                            subplot_titles=('Confusion Matrix',
                                            'Receiver operating characteristic',
                                            'Feature Importances'))
    
    fig.append_trace(trace1,1,1)
    fig.append_trace(trace2,1,2)
    fig.append_trace(trace3,1,2)
    fig.append_trace(trace4,2,1)
    
    fig['layout'].update(showlegend=False, title="Model performance" ,
                         autosize = False,height = 900,width = 800,
                         plot_bgcolor = 'rgba(240,240,240, 0.95)',
                         paper_bgcolor = 'rgba(240,240,240, 0.95)',
                         margin = dict(b = 195))
    
    fig["layout"]["xaxis2"].update(dict(title = "false positive rate"))
    fig["layout"]["yaxis2"].update(dict(title = "true positive rate"))
    fig["layout"]["xaxis3"].update(dict(showgrid = True, tickfont = dict(size = 10),
                                        tickangle = 90))
    py.iplot(fig)
    
    if threshold_plot == True : 
        visualizer = DiscriminationThreshold(algorithm)
        visualizer.fit(training_x,training_y)
        visualizer.poof()

### 4. RANDOM FOREST

Es un conjunto de arboles de decisión. Uno de los requisitos es que las features sean independientes entre sí o que tengan una correlación muy baja. Tengo que ver de corregir esto...

In [18]:
from sklearn.ensemble import RandomForestClassifier

In [19]:
#function attributes
#columns  - column used
#nf_estimators   - The number of trees in the forest.
#estimated_tree  - tree number to be displayed
#maximum_depth   - depth of the tree
#criterion_type  - split criterion type ["gini" or "entropy"]
#Model performance - prints performance of model

def plot_tree_randomforest(columns,nf_estimators,maximum_depth,
                           criterion_type,model_performance = None) :
    
    df = data_model[columns + target_col].copy()
    
    #train and test datasets
    rf_x     = df[[i for i in columns if i not in target_col + Open + Close]]
    rf_y     = df[target_col]
    
    #random forest classifier
    rfc   = RandomForestClassifier(n_estimators = nf_estimators,
                                   max_depth = maximum_depth,
                                   criterion = criterion_type)
    rfc.fit(rf_x,rf_y)
    
    if model_performance == True :
        churn_prediction(rfc, rf_x, test_X[columns], rf_y, test_Y, columns,"features",threshold_plot = False)
        
    return rfc

In [20]:
cols1 = [i for i in train_X.columns if i not in target_col + Open + Close] 

model = plot_tree_randomforest(cols1,1000,9,"entropy",True)

RandomForestClassifier(criterion='entropy', max_depth=9, n_estimators=1000)

 Classification report : 
               precision    recall  f1-score   support

       False       1.00      1.00      1.00        97
        True       1.00      1.00      1.00         3

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100

Accuracy   Score :  1.0
Area under curve (AUC) :  1.0 



In [21]:
test_Y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 171 to 48
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Return  100 non-null    bool 
dtypes: bool(1)
memory usage: 900.0 bytes


In [22]:
rf_x = test_X[[i for i in cols if i not in target_col + Open + Close]]
rf_y = model.predict(rf_x)

In [23]:
thresholds = np.arange(0,1,0.01)

In [24]:
def to_labels(pos_probs, threshold):
    return (pos_probs >= threshold).astype('int')

predicted_proba = model.predict_proba(rf_x)

# keep probabilities for the positive outcome only

predicted = predicted_proba [:,1]
scores = [metrics.f1_score(test_Y, to_labels(predicted, t)) for t in thresholds]

# get best threshold

ix = argmax(scores)

print('Threshold=%.3f, F-Score=%.5f' % (thresholds[ix], scores[ix]))

Threshold=0.240, F-Score=1.00000


In [25]:
#I define the tuned threshold parameter
threshold = ix/100
threshold

0.24

In [26]:
rf_x = test_X[[i for i in cols if i not in target_col + Open + Close]]
predicted_proba = model.predict_proba(rf_x)
predicted = (predicted_proba [:,1] >= threshold).astype('int')
#rf_y = model.predict(rf_x)
#array_2 = predicted

In [27]:
recall = metrics.recall_score(test_Y, predicted)
f1_score = metrics.f1_score(test_Y, predicted)

In [28]:
metrics.recall_score(test_Y, predicted)

1.0

In [29]:
result = data_model[-2:]

In [30]:
result

,Open_Time,Close_Time,Open_norm,High_norm,Low_norm,Close_norm,Trades_norm,Taker_buy_Volume_norm,Taker_buy_asset_volume_norm,Return
497,2022-01-19 06:00:00,2022-01-19 06:59:59.999,-1.362066,-1.323301,-1.294819,-1.274094,-0.222270,0.167897,0.056898,False
498,2022-01-19 07:00:00,2022-01-19 07:59:59.999,-1.279200,-1.337138,-1.522003,-1.303338,0.369389,0.309420,0.187881,False


In [31]:
rf_x = result[[i for i in cols if i not in target_col + Open + Close]]

In [32]:
rf_x

,Open_norm,High_norm,Low_norm,Close_norm,Trades_norm,Taker_buy_Volume_norm,Taker_buy_asset_volume_norm
497,-1.362066,-1.323301,-1.294819,-1.274094,-0.222270,0.167897,0.056898
498,-1.279200,-1.337138,-1.522003,-1.303338,0.369389,0.309420,0.187881


In [33]:
rf_y  = result[target_col]
predicted_proba = model.predict_proba(rf_x)
predicted = (predicted_proba [:,1] >= threshold).astype('int')

In [34]:
rf_y

,Return
497,False
498,False


In [35]:
predicted

array([0, 0])

In [36]:
str(predicted[0]) == 'False'

False

In [37]:
rf_x

,Open_norm,High_norm,Low_norm,Close_norm,Trades_norm,Taker_buy_Volume_norm,Taker_buy_asset_volume_norm
497,-1.362066,-1.323301,-1.294819,-1.274094,-0.222270,0.167897,0.056898
498,-1.279200,-1.337138,-1.522003,-1.303338,0.369389,0.309420,0.187881


In [38]:
predicted[0]

0

In [39]:
data_model[498:499]

,Open_Time,Close_Time,Open_norm,High_norm,Low_norm,Close_norm,Trades_norm,Taker_buy_Volume_norm,Taker_buy_asset_volume_norm,Return
498,2022-01-19 07:00:00,2022-01-19 07:59:59.999,-1.2792,-1.337138,-1.522003,-1.303338,0.369389,0.30942,0.187881,False


In [40]:
file = df[498:499]
file = file.drop(['index','Return'], axis = 1)
file['Estimation'] = np.where(predicted[0] == 0, False, True)
file['Recall'] = recall
from datetime import datetime
file['timestamp'] = datetime.now()
file['threshold'] = threshold
file['F1_Score'] = f1_score
file

,Open_Time,Open,High,Low,Close,Volume,Close_Time,Quote_Asset_Volume,Trades,Taker_buy_Volume,Taker_buy_asset_volume,Estimation,Recall,timestamp,threshold,F1_Score
498,2022-01-19 07:00:00,3086.01,3088.82,3000.0,3078.34,22051.4834,2022-01-19 07:59:59.999,6.740651e+07,36102,8428.9457,2.585609e+07,False,1.0,2022-01-19 08:44:23.060774,0.24,1.0


In [41]:
#file.to_csv('ML_DB_Binance_V3.csv',index = False)
file.to_csv('ML_DB_Binance_V3.csv', header = None, mode = 'a', index = False)

#  5. BUY AND SELL ORDERS

In [42]:
from binance.client import Client
import itertools
from itertools import product
import os
from datetime import datetime
import numpy as np
from binance.client import Client
from binance.enums import *
from binance.exceptions import BinanceAPIException, BinanceOrderException

In [122]:
# init
api_key = 'YOUR_KEY'
api_secret = 'YOUR_API_SECRET'

In [123]:
client = Client(api_key, api_secret)

In [124]:
mkt_info = client.get_orderbook_tickers()
mkt_info = pd.DataFrame(mkt_info)
mkt_info = mkt_info[mkt_info['symbol'] == 'ETHUSDT']

d = {'PAIR': 'ETHUSDT', 'COMPRA': float(mkt_info['askPrice'][0:1]),
     'VENTA': float(mkt_info['bidPrice'][0:1]),
     'SYMBOL': 'ETHUSDT', 'IN': 'USDT', 'CRYPTO': 'ETH'}

mkt_info = pd.DataFrame(data = d, index =[''])

mkt_info

,PAIR,COMPRA,VENTA,SYMBOL,IN,CRYPTO
,ETHUSDT,3034.35,3034.06,ETHUSDT,USDT,ETH


In [125]:
trade = mkt_info.copy()

In [126]:
balance = client.get_asset_balance(asset=trade['IN'][0])
balance['free']

'0.09733002'

In [127]:
trade

,PAIR,COMPRA,VENTA,SYMBOL,IN,CRYPTO
,ETHUSDT,3034.35,3034.06,ETHUSDT,USDT,ETH


In [128]:
trade['VENTA'] = float(trade['COMPRA']) * 1.015

In [129]:
trade

,PAIR,COMPRA,VENTA,SYMBOL,IN,CRYPTO
,ETHUSDT,3034.35,3079.86525,ETHUSDT,USDT,ETH


In [130]:
trade['AMOUNT_IN'] = round_decimals_down(float(balance['free'])/float(trade['COMPRA']),4)
trade = trade.drop(['SYMBOL'], axis = 1)
trade

,PAIR,COMPRA,VENTA,IN,CRYPTO,AMOUNT_IN
,ETHUSDT,3034.35,3079.86525,USDT,ETH,0.0


In [131]:
amount_out = client.get_asset_balance(asset=trade['CRYPTO'][0])
trade['AMOUNT_OUT'] = str(round_decimals_down(float(amount_out['free']),4))
trade['timestamp'] = datetime.now()
trade['DOLLARS_IN'] = round(float(trade['COMPRA'][0])*float(trade['AMOUNT_IN'][0]),2)
trade['DOLLARS_OUT'] = round(float(trade['VENTA'][0])*float(trade['AMOUNT_OUT'][0]),2)

In [132]:
trade

,PAIR,COMPRA,VENTA,IN,CRYPTO,AMOUNT_IN,AMOUNT_OUT,timestamp,DOLLARS_IN,DOLLARS_OUT
,ETHUSDT,3034.35,3079.86525,USDT,ETH,0.0,0.7969,2022-01-08 16:17:19.766685,0.0,2454.34


In [133]:
recall

1.0

In [134]:
predicted[0]

0

In [138]:
if recall > 0.9 and predicted[0] == 1: 
    try:
        buy_limit = client.create_order(
        symbol=str(trade['PAIR'][0]),
        side='BUY',
        type='LIMIT',
        timeInForce='IOC',
        quantity=str(round(float(trade['AMOUNT_IN'][0]),4)),
        price=str(round(float(trade['COMPRA'][0]),2)))

    except BinanceAPIException as e:
        # error handling goes here
        print(e)
            
    except BinanceOrderException as e:
        # error handling goes here
        print(e)
            
else:
    print("No trades")

No trades


In [139]:
time.sleep(5)

In [141]:
if float(client.get_asset_balance(asset=trade['CRYPTO'][0])['free']) > 0.01 and buy_limit['status'] == 'FILLED':
    
    amount_out = client.get_asset_balance(asset=trade['CRYPTO'][0])
    
    #4 para ETH y 6 para BTC
    trade['AMOUNT_OUT'] = str(round_decimals_down(float(amount_out['free']),4))
    
    try:
        sell_limit = client.order_oco_sell(
        symbol='ETHUSDT',
        quantity=str(trade['AMOUNT_OUT'][0]),
        price=str(round(float(trade['VENTA'][0]),2)),
        stopPrice= str(round(float(trade['COMPRA'][0])*.971,2)),                                            
        stopLimitPrice= str(round(float(trade['COMPRA'][0])*.97,2)),
        stopLimitTimeInForce='GTC')

    except BinanceAPIException as e:
        # error handling goes here
        print(e)
    except BinanceOrderException as e:
        # error handling goes here
        print(e)

    trade['DOLLARS_IN'] = round(float(trade['COMPRA'][0])*float(trade['AMOUNT_IN'][0]),2)
    trade['DOLLARS_OUT'] = round(float(trade['VENTA'][0])*float(trade['AMOUNT_OUT'][0]),2)
    trade['timestamp'] = datetime.now()
    trade['Recall'] = recall
    trade['Threshold'] = threshold
    trade['F1_Score'] = f1_score
    
    trade.to_csv('/Users/farmentano12/Google Drive/DATA SCIENCE/Proyecto_ML_Binance_V3/ML_trades_Binance_V3.csv', header = None, mode = 'a', index = False)
    
    print("Trades DONE")

else:
    print("No sell order")
    

NameError: name 'buy_limit' is not defined